In [ ]:
!pip install datasets transformers timm
!pip install kaggle --upgrade

import os
import cv2
import gc
import numpy as np
import pandas as pd
import itertools
from tqdm.autonotebook import tqdm
import albumentations as A
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
import timm
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
from torchvision import transforms
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torchvision.models import ResNet50_Weights

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/CODEBRIM/classification_dataset"

Mounted at /content/drive
Images	metadata  test	val


In [3]:
train_val_path = "/content/drive/My Drive/CODEBRIM/"
test_path = "/content/drive/My Drive/CODEBRIM/"
image_path = "/content/drive/My Drive/CODEBRIM/classification_dataset/Images"

In [5]:
train_val_path_act = "/content/drive/My Drive/CODEBRIM/train_val.csv"
df = pd.read_csv(train_val_path_act)
df
#This df has train and validation images
#rest data analysis done in other notebook

,index,image,Background,sum,id,caption
0,0,image_0000005_crop_0000001.png,0,2,0,"The photo of a concrete having Efflorescence, ..."
1,1,image_0000005_crop_0000002.png,0,2,0,"The photo of a concrete having Efflorescence, ..."
2,2,image_0000021_crop_0000001.png,0,1,0,The photo of a concrete having Efflorescence
3,3,image_0000028_crop_0000001.png,0,2,0,"The photo of a concrete having Efflorescence, ..."
4,4,image_0000028_crop_0000002.png,0,1,0,The photo of a concrete having Efflorescence
...,...,...,...,...,...,...
7090,609,image_0001243_crop_0000004.png,1,0,1418,The photo of a concrete having no faults
7091,610,image_0001243_crop_0000005.png,1,0,1418,The photo of a concrete having no faults
7092,611,image_0001280_crop_0000001.png,1,0,1418,The photo of a concrete having no faults
7093,612,image_0001280_crop_0000002.png,1,0,1418,The photo of a concrete having no faults


In [6]:
class CFG:
    debug = False
    image_path = image_path
    train_val_path = train_val_path
    test_path = test_path
    batch_size = 32
    num_workers = 2
    head_lr = 1e-3
    image_encoder_lr = 1e-4
    text_encoder_lr = 1e-5
    weight_decay = 1e-3
    patience = 1
    factor = 0.8
    epochs = 10
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model_name = 'resnet50'
    image_embedding = 2048
    text_encoder_model = "distilbert-base-uncased"
    text_embedding = 768
    text_tokenizer = "distilbert-base-uncased"
    max_length = 200

    pretrained = True # for both image encoder and text encoder
    trainable = True # for both image encoder and text encoder
    temperature = 1.0

    # image size
    size = 224

    # for projection head; used for both image and text encoders
    num_projection_layers = 1
    projection_dim = 256
    dropout = 0.1

In [7]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

In [8]:
class CLIPDataset(torch.utils.data.Dataset):
    def __init__(self, image_filenames, captions, tokenizer, transforms):
        """
        image_filenames and cpations must have the same length; so, if there are
        multiple captions for each image, the image_filenames must have repetitive
        file names
        """

        self.image_filenames = image_filenames
        self.captions = list(captions)
        self.encoded_captions = tokenizer(
            list(captions), padding=True, truncation=True, max_length=CFG.max_length
        )
        self.transforms = transforms

    def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }

        image = cv2.imread(f"{CFG.image_path}/{self.image_filenames[idx]}")
        image = self.transforms(image=image)['image']
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['caption'] = self.captions[idx]

        return item


    def __len__(self):
        return len(self.captions)



def get_transforms(mode="train"):
    if mode == "train":
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )
    else:
        return A.Compose(
            [
                A.Resize(CFG.size, CFG.size, always_apply=True),
                A.Normalize(max_pixel_value=255.0, always_apply=True),
            ]
        )

In [9]:
class ImageEncoder(nn.Module):
    """
    Encode images to a fixed size vector
    """

    def __init__(
        self, model_name=CFG.model_name, pretrained=CFG.pretrained, trainable=CFG.trainable
    ):
        super().__init__()
        self.model = timm.create_model(
            model_name, pretrained, num_classes=0, global_pool="avg"
        )
        for p in self.model.parameters():
            p.requires_grad = trainable

    def forward(self, x):
        return self.model(x)

In [10]:
class TextEncoder(nn.Module):
    def __init__(self, model_name=CFG.text_encoder_model, pretrained=CFG.pretrained, trainable=CFG.trainable):
        super().__init__()
        if pretrained:
            self.model = DistilBertModel.from_pretrained(model_name)
        else:
            self.model = DistilBertModel(config=DistilBertConfig())

        for p in self.model.parameters():
            p.requires_grad = trainable

        # we are using the CLS token hidden representation as the sentence's embedding
        self.target_token_idx = 0

    def forward(self, input_ids, attention_mask):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = output.last_hidden_state
        return last_hidden_state[:, self.target_token_idx, :]

In [11]:
class ProjectionHead(nn.Module):
    def __init__(
        self,
        embedding_dim,
        projection_dim=CFG.projection_dim,
        dropout=CFG.dropout
    ):
        super().__init__()
        self.projection = nn.Linear(embedding_dim, projection_dim)
        self.gelu = nn.GELU()
        self.fc = nn.Linear(projection_dim, projection_dim)
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(projection_dim)

    def forward(self, x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [12]:
class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=CFG.temperature,
        image_embedding=CFG.image_embedding,
        text_embedding=CFG.text_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectionHead(embedding_dim=image_embedding)
        self.text_projection = ProjectionHead(embedding_dim=text_embedding)
        self.temperature = temperature

    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)

        # Calculating the Loss
        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax(
            (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
        )
        texts_loss = cross_entropy(logits, targets, reduction='none')
        images_loss = cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()


def cross_entropy(preds, targets, reduction='none'):
    log_softmax = nn.LogSoftmax(dim=-1)
    loss = (-targets * log_softmax(preds)).sum(1)
    if reduction == "none":
        return loss
    elif reduction == "mean":
        return loss.mean()

In [13]:
def make_train_valid_dfs():
    dataframe = pd.read_csv(f"{CFG.train_val_path}/train_val.csv")
    max_id = dataframe["id"].max() + 1 if not CFG.debug else 100
    image_ids = np.arange(0, max_id)
    np.random.seed(42)
    valid_ids = np.random.choice(
        image_ids, size=int(0.2 * len(image_ids)), replace=False
    )
    train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    train_dataframe = dataframe[dataframe["id"].isin(train_ids)].reset_index(drop=True)
    valid_dataframe = dataframe[dataframe["id"].isin(valid_ids)].reset_index(drop=True)
    return train_dataframe, valid_dataframe


def build_loaders(dataframe, tokenizer, mode):
    transforms = get_transforms(mode=mode)
    dataset = CLIPDataset(
        dataframe["image"].values,
        dataframe["caption"].values,
        tokenizer=tokenizer,
        transforms=transforms,
    )
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=CFG.batch_size,
        num_workers=CFG.num_workers,
        shuffle=True if mode == "train" else False,
    )
    return dataloader

In [14]:
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter


def valid_epoch(model, valid_loader):
    loss_meter = AvgMeter()

    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    for batch in tqdm_object:
        batch = {k: v.to(CFG.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(valid_loss=loss_meter.avg)
    return loss_meter


def main():
    train_df, valid_df = make_train_valid_dfs()
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
    train_loader = build_loaders(train_df, tokenizer, mode="train")
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")


    model = CLIPModel().to(CFG.device)
    params = [
        {"params": model.image_encoder.parameters(), "lr": CFG.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": CFG.text_encoder_lr},
        {"params": itertools.chain(
            model.image_projection.parameters(), model.text_projection.parameters()
        ), "lr": CFG.head_lr, "weight_decay": CFG.weight_decay}
    ]
    optimizer = torch.optim.AdamW(params, weight_decay=0.)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=CFG.patience, factor=CFG.factor
    )
    step = "epoch"

    best_loss = float('inf')
    for epoch in range(CFG.epochs):
        print(f"Epoch: {epoch + 1}")
        model.train()
        train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
        model.eval()
        with torch.no_grad():
            valid_loss = valid_epoch(model, valid_loader)

        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            torch.save(model.state_dict(), "best.pt")
            print("Saved Best Model!")

        lr_scheduler.step(valid_loss.avg)

In [15]:
main()

Epoch: 1


  0%|          | 0/178 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

Saved Best Model!


In [96]:
def make_test_dfs():
    dataframe = pd.read_csv(f"{CFG.test_path}/test.csv")
    max_id = dataframe["id"].max() + 1 if not CFG.debug else 100
    image_ids = np.arange(0, max_id)
    test_ids = [id_ for id_ in image_ids]
    test_dataframe = dataframe[dataframe["id"].isin(test_ids)].reset_index(drop=True)
    return test_dataframe

In [97]:
test_df = make_test_dfs()

In [98]:
labels = ['Crack','Spallation','Efflorescence','ExposedBars','CorrosionStain','No faults']
clip_labels = [f"a photo of concrete having {label}" for label in labels]
clip_labels
np.shape(clip_labels)

(6,)

In [99]:
tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
model = CLIPModel().to(CFG.device)
model.load_state_dict(torch.load("best.pt", map_location=CFG.device))
model.eval()

encoded_query = tokenizer([clip_labels[0]])
batch = {
    key: torch.tensor(values).to(CFG.device)
    for key, values in encoded_query.items()
}
with torch.no_grad():
    text_features = model.text_encoder(
        input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
    )
    T1 = model.text_projection(text_features)

encoded_query = tokenizer([clip_labels[1]])
batch = {
    key: torch.tensor(values).to(CFG.device)
    for key, values in encoded_query.items()
}
with torch.no_grad():
    text_features = model.text_encoder(
        input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
    )
    T2 = model.text_projection(text_features)

encoded_query = tokenizer([clip_labels[2]])
batch = {
    key: torch.tensor(values).to(CFG.device)
    for key, values in encoded_query.items()
}
with torch.no_grad():
    text_features = model.text_encoder(
        input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
    )
    T3 = model.text_projection(text_features)

encoded_query = tokenizer([clip_labels[3]])
batch = {
    key: torch.tensor(values).to(CFG.device)
    for key, values in encoded_query.items()
}
with torch.no_grad():
    text_features = model.text_encoder(
        input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
    )
    T4 = model.text_projection(text_features)

encoded_query = tokenizer([clip_labels[4]])
batch = {
    key: torch.tensor(values).to(CFG.device)
    for key, values in encoded_query.items()
}
with torch.no_grad():
    text_features = model.text_encoder(
        input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
    )
    T5 = model.text_projection(text_features)

encoded_query = tokenizer([clip_labels[5]])
batch = {
    key: torch.tensor(values).to(CFG.device)
    for key, values in encoded_query.items()
}
with torch.no_grad():
    text_features = model.text_encoder(
        input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
    )
    T6 = model.text_projection(text_features)

In [100]:
T = torch.cat((T1, T2, T3, T4, T5, T6), 0)
T.shape

torch.Size([6, 256])

In [102]:
def get_image_embeddings(valid_df, model_path):
    tokenizer = DistilBertTokenizer.from_pretrained(CFG.text_tokenizer)
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")

    model = CLIPModel().to(CFG.device)
    model.load_state_dict(torch.load(model_path, map_location=CFG.device))
    model.eval()

    valid_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image_features = model.image_encoder(batch["image"].to(CFG.device))
            image_embeddings = model.image_projection(image_features)
            valid_image_embeddings.append(image_embeddings)
    return model, torch.cat(valid_image_embeddings)

In [103]:
model, image_embeddings = get_image_embeddings(test_df, "best.pt")

  0%|          | 0/20 [00:00<?, ?it/s]

In [104]:
image_embeddings.shape

torch.Size([630, 256])

In [105]:
test_df

,image,Background,Crack,Spallation,Efflorescence,ExposedBars,CorrosionStain,sum,id,caption
0,image_0000077_crop_0000001.png,0,0,0,1,0,1,2,0,"The photo of a concrete having Efflorescence, ..."
1,image_0000084_crop_0000001.png,0,0,0,1,0,0,1,0,The photo of a concrete having Efflorescence
2,image_0000092_crop_0000001.png,0,0,0,1,0,1,2,0,"The photo of a concrete having Efflorescence, ..."
3,image_0000119_crop_0000001.png,0,0,0,1,0,0,1,0,The photo of a concrete having Efflorescence
4,image_0000127_crop_0000001.png,0,0,0,1,0,1,2,0,"The photo of a concrete having Efflorescence, ..."
...,...,...,...,...,...,...,...,...,...,...
625,image_0001572_crop_0000004.png,1,0,0,0,0,0,0,125,The photo of a concrete having no faults
626,image_0001572_crop_0000005.png,1,0,0,0,0,0,0,125,The photo of a concrete having no faults
627,image_0001586_crop_0000001.png,1,0,0,0,0,0,0,125,The photo of a concrete having no faults
628,image_0001586_crop_0000002.png,1,0,0,0,0,0,0,125,The photo of a concrete having no faults


In [124]:
arrbase = [ [0]*6 for i in range(len(test_df))]
for i in range(len(test_df)):
  if(test_df['sum'][i]==0):
    arrbase[i][5]=1;
  else:
    arrbase[i][0]=df['Crack'][i]
    arrbase[i][1]=df['Spallation'][i]
    arrbase[i][2]=df['Efflorescence'][i]
    arrbase[i][3]=df['ExposedBars'][i]
    arrbase[i][4]=df['CorrosionStain'][i]

In [152]:
arrmain = [ [0]*6 for i in range(len(test_df))]
text_embeddings_n = F.normalize(T, p=2, dim=-1)
for i in range(len(test_df)):
    img_emb = F.normalize(image_embeddings[i], p=2, dim=-1)
    scores = text_embeddings_n @ img_emb.T
    ts=0
    for i in scores:
        ts = ts+i
    for i in range(6):
        scores[i]=(scores[i]*100)/ts
    for j in range(5):
        if(scores[j]>=16.66):
            arrmain[i][j]=1
        else:
            arrmain[i][j]=0

In [155]:
count=0
k=0;
for i in range(len(test_df)):
  tempc=0
  for j in range(6):
    if(arrmain[i][j]==arrbase[i][j]):
      tempc+=1
      k=k+1
  if(tempc>=5):
    count+=1
acc = count/630 * 100
acc

68.57142857142857

In [153]:
acc2 = k/(630*6)*100

In [154]:
acc2

75.37037037037037